In [ ]:
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

from pathlib import Path

smiles = "C=C"
method = "wB97X-3c"
n_cpus = 8

output_name = smiles.replace("[", "_").replace("]", "_").replace("=", "dbl")
output_dir = Path(f"calc/{output_name}/")
output_xyz = output_dir / Path("guess.xyz")
Path(output_dir).mkdir(parents=True, exist_ok=True)

if not Path.exists(output_xyz):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    print(type(mol))
    AllChem.EmbedMolecule(mol)

    xyz = Chem.MolToXYZBlock(mol)
    output_xyz.write_text(" " + xyz)

    natoms = mol.GetNumAtoms()
    charge = Chem.GetFormalCharge(mol)
    multiplicity = Descriptors.NumRadicalElectrons(mol) + 1

    inp_text = f"! PAL{n_cpus}\n! {method}\n! OPT\n* xyzfile {charge} {multiplicity} guess.xyz\n"

    output_inp = output_dir / Path("opt.inp")
    output_inp.write_text(inp_text)

    sh_text = (
        "#!/bin/bash\n\n"
        "#SBATCH --partition=batch\n"
        f"#SBATCH --job-name=opt\n"
        f"#SBATCH --ntasks={n_cpus}\n"
        "#SBATCH --time=4:00:00\n"
        "#SBATCH --mem-per-cpu=10G\n\n"
        "module load ORCA/6.1\n"
        f"$(which orca) opt.inp > opt.log"
    )

    output_sh = output_dir / Path("run_opt.sh")
    output_sh.write_text(sh_text)

natoms = int(output_xyz.read_text().split()[0])

viewer = py3Dmol.view()
viewer.addModel(output_xyz.read_text(), "xyz")
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})

for i in range(natoms):
    viewer.addLabel(
        i,
        {
            "backgroundOpacity": 0,
            "fontColor": "blue",
            "alignment": "center",
            "inFront": True,
        },
        {"index": i},
    )

viewer.zoomTo()
viewer.show()

<class 'rdkit.Chem.rdchem.Mol'>


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import py3Dmol

smiles = "[CH2]CCC1CO1"

output_name = smiles.replace("[", "_").replace("]", "_")
output_dir = Path(f"calc/{output_name}/")

traj = output_dir / Path("opt_trj.xyz")

viewer = py3Dmol.view()
viewer.addModelsAsFrames(traj.read_text(), "xyz")
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})
viewer.animate({"loop": "forward"})
viewer.zoomTo()
viewer.show()

15
Coordinates from ORCA-job opt E -49.831312646217
  C       2.305377     -0.085068     -0.869960
  C       0.975159     -0.540028     -0.267730
  C       0.140366      0.696749     -0.067327
  C      -1.208682      0.428285      0.520937
  C      -2.090389     -0.486698     -0.306237
  O      -1.554438     -0.853257      0.923356
  H       2.383503      1.021667     -0.915261
  H       3.141287     -0.429267     -0.212361
  H       0.492146     -1.192188     -1.010228
  H       1.149738     -1.105647      0.658222
  H       0.105161      1.215729     -1.050010
  H       0.693340      1.353101      0.634097
  H      -1.716243      1.231644      1.064872
  H      -1.656787     -1.052447     -1.157315
  H      -3.159539     -0.202576     -0.372053
15
Coordinates from ORCA-job opt E -49.846616917031
  C       2.361231     -0.090578     -0.818398
  C       1.010059     -0.515187     -0.260413
  C       0.127323      0.708683     -0.069261
  C      -1.243269      0.416487      0.484342
  C

3Dmol.js failed to load for some reason. Please check your browser console for error messages.